In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ayuraj/asl-dataset

Dataset URL: https://www.kaggle.com/datasets/ayuraj/asl-dataset
License(s): CC0-1.0
  0% 0.00/56.9M [00:00<?, ?B/s]
100% 56.9M/56.9M [00:00<00:00, 1.38GB/s]


In [ ]:
import zipfile
with zipfile.ZipFile("asl-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/asl_dataset")

In [ ]:
import shutil

# Path to the unwanted nested folder
unwanted_path = "/content/asl_dataset/asl_dataset/asl_dataset"

# Remove it
shutil.rmtree(unwanted_path)
print("Unwanted folder deleted.")


Unwanted folder deleted.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
dataset_path = "/content/asl_dataset/asl_dataset"

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

In [ ]:
test_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),  # MobileNetV2 requirement
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 2012 images belonging to 36 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 503 images belonging to 36 classes.


In [ ]:
num_classes = train_generator.num_classes

In [ ]:
from tensorflow.keras.applications import MobileNetV2
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("best_model.h5", save_best_only=True)
]

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)


model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,304,100 (8.79 MB)

 Trainable params: 1,572,516 (6.00 MB)

 Non-trainable params: 731,584 (2.79 MB)

In [ ]:
history=model.fit(
    train_generator,
    epochs=30,
    validation_data=test_generator,
    callbacks=callbacks
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step - accuracy: 0.0363 - loss: 3.8829

63/63 ━━━━━━━━━━━━━━━━━━━━ 56s 617ms/step - accuracy: 0.0366 - loss: 3.8803 - val_accuracy: 0.0557 - val_loss: 3.7681
Epoch 2/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step - accuracy: 0.1402 - loss: 3.2570

63/63 ━━━━━━━━━━━━━━━━━━━━ 25s 396ms/step - accuracy: 0.1406 - loss: 3.2554 - val_accuracy: 0.0994 - val_loss: 3.5216
Epoch 3/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step - accuracy: 0.2501 - loss: 2.8399

63/63 ━━━━━━━━━━━━━━━━━━━━ 25s 401ms/step - accuracy: 0.2507 - loss: 2.8380 - val_accuracy: 0.1133 - val_loss: 3.3049
Epoch 4/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step - accuracy: 0.3730 - loss: 2.4298

...

63/63 ━━━━━━━━━━━━━━━━━━━━ 25s 399ms/step - accuracy: 0.9291 - loss: 0.3183 - val_accuracy: 0.8608 - val_loss: 0.3947
Epoch 27/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step - accuracy: 0.9252 - loss: 0.3146

63/63 ━━━━━━━━━━━━━━━━━━━━ 25s 399ms/step - accuracy: 0.9251 - loss: 0.3147 - val_accuracy: 0.8708 - val_loss: 0.3734
Epoch 28/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 380ms/step - accuracy: 0.9201 - loss: 0.3222

63/63 ━━━━━━━━━━━━━━━━━━━━ 26s 407ms/step - accuracy: 0.9201 - loss: 0.3218 - val_accuracy: 0.8767 - val_loss: 0.3552
Epoch 29/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step - accuracy: 0.9450 - loss: 0.2700

63/63 ━━━━━━━━━━━━━━━━━━━━ 25s 400ms/step - accuracy: 0.9449 - loss: 0.2700 - val_accuracy: 0.8807 - val_loss: 0.3428
Epoch 30/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step - accuracy: 0.9350 - loss: 0.2769

63/63 ━━━━━━━━━━━━━━━━━━━━ 25s 394ms/step - accuracy: 0.9350 - loss: 0.2767 - val_accuracy: 0.8847 - val_loss: 0.3342


In [ ]:
import json

# Save mapping
class_indices = train_generator.class_indices  # e.g., {'a': 0, 'b': 1, ..., 'z': 25}
index_to_class = {v: k for k, v in class_indices.items()}

# Save to JSON
with open("class_mapping.json", "w") as f:
    json.dump(index_to_class, f)

print("Saved mapping to class_mapping.json")


Saved mapping to class_mapping.json
